In [194]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import re
pd.options.display.max_rows = 4000

In [195]:
df = pd.read_csv('./canadian-hospital-re-admittance-challenge/train.csv')
test_df = pd.read_csv('./canadian-hospital-re-admittance-challenge/test.csv')
df

,enc_id,patient_id,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmission_id
0,88346340,2488608,Caucasian,Male,[60-70),NaN,1,2,6,3,...,No,Steady,No,No,No,No,No,Ch,Yes,2
1,92001408,52133202,Caucasian,Male,[70-80),[100-125),2,6,1,7,...,No,No,No,No,No,No,No,No,Yes,1
2,169424316,40945509,Caucasian,Female,[70-80),NaN,3,2,1,7,...,No,Up,No,No,No,No,No,Ch,Yes,1
3,272987082,38850777,Caucasian,Female,[50-60),NaN,1,1,7,1,...,No,No,No,No,No,No,No,No,Yes,2
4,150600612,72738225,Caucasian,Female,[80-90),NaN,1,6,7,6,...,No,Down,No,No,No,No,No,Ch,Yes,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,198619164,85063725,Caucasian,Male,[70-80),NaN,1,1,7,6,...,No,No,No,No,No,No,No,No,Yes,1
71232,177404100,86244345,Caucasian,Male,[90-100),NaN,1,3,7,5,...,No,No,No,No,No,No,No,No,No,2
71233,50905206,5131368,Caucasian,Male,[70-80),NaN,3,6,1,6,...,No,Steady,No,No,No,No,No,Ch,Yes,2
71234,216431502,85969035,Hispanic,Male,[50-60),NaN,1,1,4,4,...,No,Steady,No,No,No,No,No,No,Yes,2


In [196]:
percent_missing = df.isnull().sum() * 100 / len(df)

percentages_df = pd.DataFrame({
    'percent_missing': percent_missing
})

percentages_df

,percent_missing
enc_id,0.000000
patient_id,0.000000
race,2.275535
gender,0.000000
age,0.000000
weight,96.841485
admission_type_id,0.000000
discharge_disposition_id,0.000000
admission_source_id,0.000000
time_in_hospital,0.000000


In [197]:
df.drop(columns=['weight','payer_code','max_glu_serum','A1Cresult'], inplace=True)
test_df.drop(columns=['weight','payer_code','max_glu_serum','A1Cresult'],inplace=True)

# df["medical_specialty"].fillna("No-Admitting-Physician", inplace=True)
# test_df["medical_specialty"].fillna("No-Admitting-Physician", inplace=True)

df

,enc_id,patient_id,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmission_id
0,88346340,2488608,Caucasian,Male,[60-70),1,2,6,3,Family/GeneralPractice,...,No,Steady,No,No,No,No,No,Ch,Yes,2
1,92001408,52133202,Caucasian,Male,[70-80),2,6,1,7,NaN,...,No,No,No,No,No,No,No,No,Yes,1
2,169424316,40945509,Caucasian,Female,[70-80),3,2,1,7,NaN,...,No,Up,No,No,No,No,No,Ch,Yes,1
3,272987082,38850777,Caucasian,Female,[50-60),1,1,7,1,NaN,...,No,No,No,No,No,No,No,No,Yes,2
4,150600612,72738225,Caucasian,Female,[80-90),1,6,7,6,NaN,...,No,Down,No,No,No,No,No,Ch,Yes,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,198619164,85063725,Caucasian,Male,[70-80),1,1,7,6,NaN,...,No,No,No,No,No,No,No,No,Yes,1
71232,177404100,86244345,Caucasian,Male,[90-100),1,3,7,5,NaN,...,No,No,No,No,No,No,No,No,No,2
71233,50905206,5131368,Caucasian,Male,[70-80),3,6,1,6,Cardiology,...,No,Steady,No,No,No,No,No,Ch,Yes,2
71234,216431502,85969035,Hispanic,Male,[50-60),1,1,4,4,InternalMedicine,...,No,Steady,No,No,No,No,No,No,Yes,2


In [198]:
df.drop(columns=['race','gender','admission_type_id','admission_source_id','diag_1','diag_2','diag_3'], inplace=True)
test_df.drop(columns=['race','gender','admission_type_id','admission_source_id','diag_1','diag_2','diag_3'], inplace=True)
df

,enc_id,patient_id,age,discharge_disposition_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmission_id
0,88346340,2488608,[60-70),2,3,Family/GeneralPractice,54,3,10,0,...,No,Steady,No,No,No,No,No,Ch,Yes,2
1,92001408,52133202,[70-80),6,7,NaN,50,6,35,2,...,No,No,No,No,No,No,No,No,Yes,1
2,169424316,40945509,[70-80),2,7,NaN,49,4,20,2,...,No,Up,No,No,No,No,No,Ch,Yes,1
3,272987082,38850777,[50-60),1,1,NaN,1,1,15,0,...,No,No,No,No,No,No,No,No,Yes,2
4,150600612,72738225,[80-90),6,6,NaN,58,2,26,1,...,No,Down,No,No,No,No,No,Ch,Yes,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,198619164,85063725,[70-80),1,6,NaN,61,0,14,0,...,No,No,No,No,No,No,No,No,Yes,1
71232,177404100,86244345,[90-100),3,5,NaN,36,0,12,2,...,No,No,No,No,No,No,No,No,No,2
71233,50905206,5131368,[70-80),6,6,Cardiology,62,6,28,0,...,No,Steady,No,No,No,No,No,Ch,Yes,2
71234,216431502,85969035,[50-60),1,4,InternalMedicine,29,1,11,0,...,No,Steady,No,No,No,No,No,No,Yes,2


In [199]:
for col in df.loc[:,'metformin':'diabetesMed']:
    med_groups = df.groupby(by=[col])
    print(med_groups.count().iloc[:, 0])

metformin
Down        392
No        57223
Steady    12885
Up          736
Name: enc_id, dtype: int64
repaglinide
Down         33
No        70145
Steady      980
Up           78
Name: enc_id, dtype: int64
nateglinide
Down          8
No        70770
Steady      444
Up           14
Name: enc_id, dtype: int64
chlorpropamide
No        71170
Steady       62
Up            4
Name: enc_id, dtype: int64
glimepiride
Down        131
No        67558
Steady     3308
Up          239
Name: enc_id, dtype: int64
acetohexamide
No        71235
Steady        1
Name: enc_id, dtype: int64
glipizide
Down        376
No        62301
Steady     8011
Up          548
Name: enc_id, dtype: int64
glyburide
Down        389
No        63713
Steady     6577
Up          557
Name: enc_id, dtype: int64
tolbutamide
No        71221
Steady       15
Name: enc_id, dtype: int64
pioglitazone
Down         83
No        66074
Steady     4910
Up          169
Name: enc_id, dtype: int64
rosiglitazone
Down         67
No        66740
Stea

In [200]:
df.drop(columns=['chlorpropamide', 'tolbutamide', 'miglitol', 'acarbose', 'tolazamide', 'acetohexamide', 'troglitazone', 'examide', 'citoglipton', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'glyburide-metformin'], inplace=True)
test_df.drop(columns=['chlorpropamide', 'tolbutamide', 'miglitol', 'acarbose', 'tolazamide', 'acetohexamide', 'troglitazone', 'examide', 'citoglipton', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'glyburide-metformin'], inplace=True)
df.columns.size

25

In [201]:
def count_changes(row):
    count =0
    for col in ['metformin','repaglinide','nateglinide','glimepiride','glipizide','glyburide','pioglitazone','rosiglitazone','insulin']:
        if(row[col]=='Up' or row[col]=='Down'):
            count+=1
    return count

new_col = df.apply(count_changes, axis=1)
df.insert(loc = len(df.columns)-1, column = 'changes', value=new_col)

new_col = test_df.apply(count_changes, axis=1)
test_df.insert(loc = len(test_df.columns), column = 'changes', value=new_col)

In [202]:
df.drop(columns=['metformin','repaglinide','nateglinide','glimepiride','glipizide','glyburide','pioglitazone','rosiglitazone','insulin','change'], inplace=True)
test_df.drop(columns=['metformin','repaglinide','nateglinide','glimepiride','glipizide','glyburide','pioglitazone','rosiglitazone','insulin','change'], inplace=True)

df.columns.size

16

In [203]:
# plt.figure(figsize=(20,20))
# corr = df.loc[:, ["time_in_hospital","num_lab_procedures","num_procedures","num_medications","number_outpatient","number_emergency","number_inpatient","number_diagnoses","changes", "readmission_id"]].corr()
# sns.heatmap(corr,annot=True)

In [204]:
df.drop(columns=['num_procedures','num_medications','time_in_hospital'])
test_df.drop(columns=['num_procedures','num_medications','time_in_hospital'])

,enc_id,patient_id,age,discharge_disposition_id,medical_specialty,num_lab_procedures,number_outpatient,number_emergency,number_inpatient,number_diagnoses,diabetesMed,changes
0,86305392,23846112,[40-50),5,NaN,25,0,0,4,9,Yes,0
1,394919696,90620001,[80-90),3,Emergency/Trauma,59,0,1,0,9,Yes,1
2,164917446,25342173,[60-70),1,InternalMedicine,47,0,0,0,9,Yes,1
3,178319040,23481783,[70-80),3,InternalMedicine,46,0,2,7,9,Yes,1
4,253585416,86500791,[70-80),6,NaN,64,0,0,1,9,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...
30525,169542282,23401863,[70-80),6,InternalMedicine,44,0,2,0,5,Yes,0
30526,37906254,12267054,[50-60),18,NaN,19,0,0,0,8,No,0
30527,242799822,98403552,[70-80),3,Emergency/Trauma,73,0,0,1,9,Yes,1
30528,204193212,87646410,[80-90),3,NaN,63,0,0,1,9,Yes,0


In [205]:
df.columns

Index(['enc_id', 'patient_id', 'age', 'discharge_disposition_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'diabetesMed', 'changes', 'readmission_id'],
      dtype='object')

In [206]:
test_df.columns

Index(['enc_id', 'patient_id', 'age', 'discharge_disposition_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'diabetesMed', 'changes'],
      dtype='object')

In [207]:
# index_vals = df['readmission_id'].astype('category').cat.codes

# dimensions = []

# cols = ['num_lab_procedures','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'changes']

# for i in cols:
#     d = dict(label=i, values=df[i])
#     dimensions.append(d)

# fig = go.Figure(data=go.Splom(
#      dimensions=dimensions,
#      diagonal_visible=False,
#      text=df['readmission_id'],
#      marker=dict(color=index_vals,
#                line_color='white', line_width=0.5)
# ))

# fig.update_layout(
#     title='Readmission id',
#     dragmode='select',
#     width=1300,
#     height=1300,
#     hovermode='closest',
# )

# fig.show()


In [208]:
input = df.loc[:, "enc_id":"changes"]
labels = df.loc[:, "readmission_id"]
input.columns

Index(['enc_id', 'patient_id', 'age', 'discharge_disposition_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       'diabetesMed', 'changes'],
      dtype='object')

In [209]:
input_encoded = pd.get_dummies(input, columns=['age','medical_specialty','diabetesMed','discharge_disposition_id'])

# input_encoded = pd.get_dummies(input, columns=['race', 'gender', 'age',
#        'medical_specialty', 'max_glu_serum', 'A1Cresult', 'diabetesMed', 'Number_of_Diagnosis',
#        'admission_type_id_new', 'discharge_type_id_new', 'admission_source_id_new'])

print(input_encoded.columns)

Index(['enc_id', 'patient_id', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       ...
       'discharge_disposition_id_17', 'discharge_disposition_id_18',
       'discharge_disposition_id_19', 'discharge_disposition_id_20',
       'discharge_disposition_id_22', 'discharge_disposition_id_23',
       'discharge_disposition_id_24', 'discharge_disposition_id_25',
       'discharge_disposition_id_27', 'discharge_disposition_id_28'],
      dtype='object', length=117)


In [210]:
test_encoded = pd.get_dummies(test_df, columns=['age', 'medical_specialty', 'diabetesMed', 'discharge_disposition_id'])

# test_encoded = pd.get_dummies(test_df, columns=['race', 'gender', 'age',
#        'medical_specialty', 'max_glu_serum', 'A1Cresult', 'diabetesMed', 'Number_of_Diagnosis',
#        'admission_type_id_new', 'discharge_type_id_new', 'admission_source_id_new'])


print(test_encoded.columns)

Index(['enc_id', 'patient_id', 'time_in_hospital', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'number_diagnoses',
       ...
       'discharge_disposition_id_17', 'discharge_disposition_id_18',
       'discharge_disposition_id_19', 'discharge_disposition_id_20',
       'discharge_disposition_id_22', 'discharge_disposition_id_23',
       'discharge_disposition_id_24', 'discharge_disposition_id_25',
       'discharge_disposition_id_27', 'discharge_disposition_id_28'],
      dtype='object', length=112)


In [211]:
for i in input_encoded.columns:
    if i not in test_encoded.columns:
       test_encoded[i] = 0

for i in test_encoded.columns:
    if i not in input_encoded.columns:
       test_encoded.drop(columns=[i], inplace=True)

input_encoded.sort_index(axis=1, inplace=True)
test_encoded.sort_index(axis=1, inplace=True)

In [212]:
print(input_encoded.columns)
print(test_encoded.columns)

Index(['age_[0-10)', 'age_[10-20)', 'age_[20-30)', 'age_[30-40)',
       'age_[40-50)', 'age_[50-60)', 'age_[60-70)', 'age_[70-80)',
       'age_[80-90)', 'age_[90-100)',
       ...
       'medical_specialty_Urology', 'num_lab_procedures', 'num_medications',
       'num_procedures', 'number_diagnoses', 'number_emergency',
       'number_inpatient', 'number_outpatient', 'patient_id',
       'time_in_hospital'],
      dtype='object', length=117)
Index(['age_[0-10)', 'age_[10-20)', 'age_[20-30)', 'age_[30-40)',
       'age_[40-50)', 'age_[50-60)', 'age_[60-70)', 'age_[70-80)',
       'age_[80-90)', 'age_[90-100)',
       ...
       'medical_specialty_Urology', 'num_lab_procedures', 'num_medications',
       'num_procedures', 'number_diagnoses', 'number_emergency',
       'number_inpatient', 'number_outpatient', 'patient_id',
       'time_in_hospital'],
      dtype='object', length=117)


In [213]:
scaler = StandardScaler()
input_encoded[['num_lab_procedures','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'changes']] = scaler.fit_transform(input_encoded[['num_lab_procedures','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'changes']].to_numpy())
test_encoded[['num_lab_procedures','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'changes']] = scaler.fit_transform(test_encoded[['num_lab_procedures','number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'changes']].to_numpy())

In [214]:
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('transformer', PolynomialFeatures(degree=2, include_bias=False), ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']),
#     ],
#     remainder='passthrough'
# )

# input_encoded = preprocessor.fit_transform(input_encoded)
# test_encoded = preprocessor.transform(test_encoded)

In [215]:
X_train,X_test,Y_train,Y_test = train_test_split(input_encoded, labels, test_size=0.2, random_state=42)

In [216]:
# lr = LogisticRegression(random_state=42, multi_class="multinomial")
# lr.fit(X_train,Y_train)

# y_pred = lr.predict(X_test)
# print(accuracy_score(y_pred, Y_test))

# nb = GaussianNB()
# nb.fit(X_train,Y_train)

# y_pred = nb.predict(X_test)
# print(accuracy_score(y_pred, Y_test))

# tree = DecisionTreeClassifier(max_depth=20,random_state=42)
# tree.fit(X_train,Y_train)

# y_pred = tree.predict(X_test)
# print(accuracy_score(y_pred, Y_test))

In [217]:
rf = RandomForestClassifier(random_state=42, criterion='gini', max_depth=21, n_estimators=128)
rf.fit(X_train,Y_train)

y_pred = rf.predict(X_test)
print(accuracy_score(y_pred, Y_test))

0.5920830993823695


In [218]:
# gbc = GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=4,random_state=42)
# gbc.fit(X_train,Y_train)

# y_pred = gbc.predict(X_test)
# print(accuracy_score(y_pred, Y_test))

In [219]:
test_Y = rf.predict(test_encoded)

df_output = pd.read_csv("./canadian-hospital-re-admittance-challenge/sample_submission.csv")
df_output["readmission_id"] = test_Y
df_output.to_csv("submission.csv", index=False)

In [220]:
# def objective(trial):
#     criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
#     max_depth = trial.suggest_int("max_depth", 2, 32, log=True)
#     n_estimators = trial.suggest_int("n_estimators", 100,500)
#     rf = RandomForestClassifier(criterion =criterion,
#             max_depth=max_depth, 
#             n_estimators=n_estimators
#         )
#     X_train,X_test,Y_train,Y_test = train_test_split(input_encoded, labels, test_size=0.2, random_state=42)
#     rf.fit(X_train,Y_train)
#     y_pred = rf.predict(X_test)
#     score = accuracy_score(y_pred, Y_test)
#     return score


# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=15)